<a href="https://colab.research.google.com/github/gala127/My_Important_Dates/blob/master/SoundWork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import librosa

audio_file = 'path_to_audio_file.wav'
y, sr = librosa.load(audio_file)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('path_to_pretrained_model.h5')

In [ ]:
import sounddevice as sd

def audio_callback(indata, frames, time, status):
    if status:
        print(status)
    mfccs = librosa.feature.mfcc(y=indata[:, 0], sr=sr, n_mfcc=13)
    mfccs = mfccs.reshape(1, -1)
    prediction = model.predict(mfccs)
    print(f'Predicted emotion: {prediction}')

with sd.InputStream(callback=audio_callback):
    sd.sleep(10000)  # Listen for 10 seconds


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten

model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(n_mfcc, 1)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(8, activation='softmax'))  # Assuming 8 emotion classes

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
import sounddevice as sd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import librosa

# Load your pre-trained model
model = load_model('path_to_your_russian_emotion_model.h5')

def extract_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

def audio_callback(indata, frames, time, status):
    if status:
        print(status)
    # Reshape input data for model prediction
    audio_data = np.mean(indata, axis=1)
    features = extract_features(audio_data, sr)
    features = np.expand_dims(features, axis=0)
    prediction = model.predict(features)
    emotion = np.argmax(prediction)
    print(f'Predicted emotion: {emotion}')

# Start streaming audio
sr = 16000  # Sample rate
with sd.InputStream(callback=audio_callback, channels=1, samplerate=sr):
    sd.sleep(10000)  # Listen for 10 seconds
